# Despacho múltiple, tipos abstractos y duck typing

[https://docs.julialang.org/en/v1/manual/methods/](https://docs.julialang.org/en/v1/manual/methods/)

[https://www.oxinabox.net/2020/02/09/whycompositionaljulia.html#multiple-dispatch--duck-typing](https://www.oxinabox.net/2020/02/09/whycompositionaljulia.html#multiple-dispatch--duck-typing)

## Despacho múltiple

En Julia, las siguientes definiciones son simultaneamente posibles.

In [ ]:
f(x::Int,y::Int) = println("x e y son enteros")
f(x::Real,y::Int) = println("x es real e y es entero")
f(x::Int,y::Real) = println("x es entero e y es real")
f(x::Real,y::Real) = println("x e y son reales")

Cada una de ella define un *método* de la función *genérica* `f`.

Durante un llamado a `f`, Julia selecciona el método a evaluar de acuerdo a los tipos de los argumentos especificados.

In [ ]:
f(3,4)

In [ ]:
f(π,4)

In [ ]:
f(3,√2)

In [ ]:
f(3.3,π)

Esto se llama **despacho múltiple** (multiple dispatch). Si queremos ver los métodos de una función, escribimos:

In [ ]:
methods(f)

In [ ]:
methods(cos)

### La expresividad del despacho multiple

| dispatch degree | syntax | dispatch arguments | expressive order | expressive power | lenguaje |
|--|--|--|--|--|--|
| none | $f(x_1$,$x_2$,$\dots)$ | {} | $\mathcal{O}(1)$ | constant | C, FORTRAN (< 2003) |
| single | $x_1$.$f(x_2$,$\dots)$ | {$x_1$} | $\mathcal{O}(|X_1|)$ | linear | C++, FORTRAN (2003), Java |
| multiple | $f(x_1$,$x_2$,$\dots)$ | {$x_1$,$x_2$,...}    | $\mathcal{O}(|X_1||X_2|\dots)$ | exponential | Julia, C# (4.0), LISP |

### Inferencia de tipos, compilación AOT y JIT

Para evaluar una expresión, Julia primero busca determinar la composición de métodos a evaluar en tiempo de compilación. En este sentido, Julia aplica *Ahead of Time (AOT) compilation*.

Cuando el tipo de algunos de los objetos involucrados se definen en tiempos de ejecución, AOT no puede aplicarse. Entonces, Julia utiliza un algoritmo de *infierencia de tipos* (type inference) para determinar la composición de métodos en tiempo de ejecución utilizando *Just In Time (JIT) compilation*.

## Tipos concretos y abstractos

El despacho múltiple es una herramienta poderosa si se lo combina con la noción de *tipos abstractos* (abstract types), los cuales sirven para definir jerarquías de tipos, subtipos, subsubtipos, etc.

Los *tipos concretos* no poseen subtipos y varios de ellos vienen predefinidos (`Int64`, `Float64`,`String`, etc.). 

Para tener una idea de como funciona múltiple dispatch, consideremos el siguiente ejemplo:

* [https://www.youtube.com/watch?v=kc9HwsxE1OY](https://www.youtube.com/watch?v=kc9HwsxE1OY)

In [ ]:
abstract type Pet end

struct Dog <: Pet
    name::String
end

struct Cat <: Pet
    name::String
end

function encounter(a::Pet,b::Pet)
    verb = meets(a,b) 
    println("$(a.name) meets $(b.name) and $verb.")
end

meets(a::Dog,b::Dog) = "sniff"
meets(a::Dog,b::Cat) = "chases"
meets(a::Cat,b::Dog) = "hisses"
meets(a::Cat,b::Cat) = "slinks"

In [ ]:
fido = Dog("Fido")
rex = Dog("Rex")
whiskers = Cat("Whiskers")
spots = Cat("Spots")

encounter(fido,rex)
encounter(fido,whiskers)
encounter(rex,spots)
encounter(whiskers,spots)

## Duck typing

Consideremos el siguiente ejemplo:

* [https://www.oxinabox.net/2020/02/09/whycompositionaljulia.html#multiple-dispatch--duck-typing](https://www.oxinabox.net/2020/02/09/whycompositionaljulia.html#multiple-dispatch--duck-typing)

* [https://www.compart.com/en/unicode/U+1F986](https://www.compart.com/en/unicode/U+1F986)

Supongamos que alguna libreria escrita por terceros implementa el tipo `Duck` (pato) y algunas de sus funcionalidades.

In [ ]:
struct Duck end

draw(duck) = "🦆"
function talk(duck)  
    println(draw(duck)," quack")
end
function raise_young(duck,babe) 
    println(draw(duck)," leads 🐤") 
end

In [ ]:
talk(Duck())

In [ ]:
raise_young(Duck(),Duck())

Nos interesa extender la librería (sin tocarla) agregando el tipo `Swan` (cisne).

In [ ]:
struct Swan end

In [ ]:
talk(Swan())

Esto no está bien!

Para corregir esto hacemos un poco de *duck typing*.

In [ ]:
draw(swan::Swan) = "🦢"
function talk(swan::Swan)
    println(show(swan)," hiss")
end

In [ ]:
talk(Swan())

Ahora esta mejor.

Probemos ver que pasa cuando hay cisnes bebes

In [ ]:
raise_young(Swan(),Swan())

Vemos que surge otro problema. Los cisnes no caminan delante de sus pequeños, los llevan a cococho!

Para corregirlo, escribimos

In [ ]:
function raise_young(swan::Swan,babe) 
    println(draw(swan)," carry 🐤")
end

y volvemos a probar

In [ ]:
raise_young(Swan(),Swan())

Que pasa si mezclamos especies?

In [ ]:
raise_young(Duck(),Swan())

De nuevo vemos un problema. Sabemos que los patos abandonan a los cisnes bebés.

Corregimos el problema, escribiendo

In [ ]:
function raise_young(swan::Duck,babe::Swan)
    println(draw(swan)," abandon 🐤")
end

y volvemos a probar

In [ ]:
raise_young(Duck(),Swan())

Que pasa si invertimos el orden y pedimos que un cisne críe un patito?

In [ ]:
raise_young(Swan(),Duck())

## El problema de la extensibilidad

*The Expression Problem Revisited, M. Torgersen (2004)*

En que grado nuestro paquete de código puede ser extructurado de manera tal que:

1. el **modelo de datos** y

2. el conjunto de **operaciones (virtuales)** sobre ellos

puedan ser **extendidos** sin necesidad de:

1. modificar el código ya existente,

2. repetir código y

3. sin introducir errores de tipo en tiempos de ejecución?

#### En otras palabras

Se puede fácil y "correctamente" hacer las siguientes cosas:

1. definir **tipos nuevos** sobre los cuales las **operaciones existentes** apliquen, y

2. definir **nuevas operaciones** que apliquen sobre los **tipos existentes**?

Este tipo de requermientos resultan fáciles/díficiles de implementar según el paradigma utilizado:

1. es **fácil** en lenguajes **orientados a objetos** y **difícil** en **lenguajes funcionales**, y

2. es **difícil** en lenguajes **orientados a objetos** y **fácil** en **lenguajes funcionales**.

#### Despacho múltiple propone lo mejor de ambos mundos:

1. Agregar nuevos métodos a las **funciones/operaciones genéricas existentes** para actuar sobre **tipos nuevos**.

2. Agregar nuevos métodos a **nuevas funciones/operaciones genéricas** que actúen sobre **tipos existentes**.

### Ejemplo real: DifferentialEquations + Measurements

Los paquetes `DifferentialEquations` y `Measurements` fueron implementados de manera independiente. El primero se desarrolló para resolver ecuaciones diferenciales, el segundo para lidiar con datos que tengan incerteza experimental (ej. $x = 1.35 \pm 0.1$). 

Como el paquete `Measurements` es un subtipo de `Number`, incorporando métodos nuevos para modelar operaciones matemáticas con ellos (ej. propagación de errores), el integrador ODE proveido en `DifferentialEquations` puede aplicarse directamente a mediciones:

In [ ]:
using DifferentialEquations
using Measurements

In [ ]:
g = 9.79 ± 0.02
L = 1.0 ± 0.01

u₀ = [0 ± 0, π/60 ± 0.01]
tspan = (0.0,6.3)

function pendulum(du,u,p,t)
    θ = u[1] 
    dθ = u[2]
    du[1] = dθ
    du[2] = -(g/L)*θ
end

prop = ODEProblem(pendulum, u₀, tspan)
sol = solve(prop, Tsit5(), reltol = 1e-6)

In [ ]:
using Plots

In [ ]:
sol_u2 = reduce(hcat,sol.u)'[:,2]

In [ ]:
using LaTeXStrings
scatter(sol.t,sol_u2,label="",xlabel=L"t",ylabel=L"u_2")

In [ ]:
methods(+)